In [1]:
from sleepmate.agent import *

In [11]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

In [2]:
memory = X.load_memory(max_token_limit=10, memory_key="history")

In [3]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [4]:
memory.load_memory_variables({})

{'history': [AIMessage(content='not much')]}

In [8]:
from langchain.llms import OpenAI

llm = OpenAI()

In [9]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. What about you?"

In [10]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Just working on writing some documentation!
AI:

> Finished chain.


' That sounds like a very important task. What kind of documentation are you writing?'

In [7]:
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


'Yes, I am familiar with LangChain. It is a blockchain-based language learning platform that aims to connect language learners and native speakers for interactive language practice and cultural exchange. The platform utilizes blockchain technology to provide secure and transparent transactions between users. It also incorporates features such as personalized language learning plans, language assessment tools, and a community forum for language learners to connect and support each other.'